In [1]:
!pip install haversine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import json
import psycopg2
from datetime import datetime, date
import math
import numpy as np
from haversine import haversine
from itertools import combinations

In [2]:
def load_db():
  with open("/content/drive/MyDrive/yeoreodigm/data_files/db_info.json") as json_file:
    DB_INFO = json.load(json_file)
  
  endpoint = DB_INFO["ENDPOINT"]
  dbname = DB_INFO["DB_NAME"]
  user = DB_INFO["USER_ID"]
  password = DB_INFO["PASSWORD"]
  db = psycopg2.connect(host=endpoint,dbname=dbname,user=user,password=password)
  return db

In [3]:
def load_places_location(db, place_list):
  cursor = db.cursor()
  place_list = str(tuple(place_list))
  sql = f"select place_id, latitude, longitude from places where place_id in {place_list}"
  cursor.execute(sql)
  result = cursor.fetchall()
  result = list(map(list,result))
  return result

In [4]:
def calculate_distance_haversine(A,B):
  A_x = A[1]
  A_y = A[2]
  B_x = B[1]
  B_y = B[2]
  dist = haversine((A_x,A_y),(B_x,B_y))
  return dist

In [5]:
def make_dist_matrix_haversine(location_info):
  print("making matrix!!!")
  matrix_size = len(location_info)
  #initialize empty n*n matrix
  dist_matrix = [([0] * matrix_size) for _ in range(matrix_size)]
  for i in range(matrix_size):
    for j in range(i+1,matrix_size):
      dist_matrix[i][j] = calculate_distance_haversine(location_info[i],location_info[j])
      dist_matrix[j][i] = dist_matrix[i][j]
  return dist_matrix

In [21]:
def optimize_course1(place_list,day,db):
  place_info = load_places_location(db, place_list)
  save_id = []  
  result = [[] for _ in range(day)]
  
  AIRPORT = (33.5059364,126.4959513)
  length = len(place_list)
  total_dist = [0]*length
  visit_result = [[] for _ in range(length)]

  dist_matrix = make_dist_matrix_haversine(place_info)

  for i in range(length):
    start = i
    end = 999
    visit = [False] * length
    visit[i] = True    
    now = i
    min_dist = 100
    min_idx = 999
    visit_result[i].append(now)
    print("start : ",start)
    while False in visit: #모든 클러스터를 방문할때까지
      for idx, dist in enumerate(dist_matrix[now]):
        if visit[idx] == True:
          continue
        else:
          if dist < min_dist:
            min_idx = idx
            min_dist = dist
      print("dist from %d to %d : %.2f  "%(now,min_idx,min_dist))
      
      total_dist[start] += min_dist
      now = min_idx
      visit[now] = True
      visit_result[start].append(now)
      min_dist=999
      end = min_idx

    airport_to_start = haversine(AIRPORT,place_info[start][1:]) # 공항에서 시작지점 까지 거리
    end_to_airport = haversine(AIRPORT,place_info[end][1:]) #마지막지점에서 공항 가는 거리
    total_dist[i] += airport_to_start 
    total_dist[i] += end_to_airport  

    print("airport - 시작지점(%s) 까지 거리%s 합산"%(start,airport_to_start))
    print("끝지점(%s) - 공항  까지 거리%s 합산"%(end,end_to_airport))
    print("total_dist : ",total_dist[i])

  shortest_cluster = np.array(total_dist).argsort()[0]
  result_index = visit_result[shortest_cluster]  #최단거리를 보장하는 index 방문 순서 마지막에 이 인덱스랑 Id랑 매핑해줘야함.
 

  #day dividing ...
  day_dividing = []
  print("result_index: ",result_index)
  for comb in combinations(result_index,day-1):
    #comb : (5,3,7)
    #print("comb : ",comb)
    if comb[-1] == result_index[-1]: 
      continue
    dist = 0
    for now in comb:
      next = result_index[result_index.index(now) + 1]
      now_loc = place_info[now][1:]
      next_loc = place_info[next][1:]
      
      dist += haversine(now_loc,next_loc)
    day_dividing.append((dist,comb))
    
  day_dividing.sort()
  divider = day_dividing[-1][1]
  print("divider : ",divider)
  final_result = []
  today_places = []
  start = 0
  for i in divider:
    cut = result_index.index(i)  
    final_result.append(result_index[start:cut+1])
    start = cut+1
  final_result.append(result_index[start:])
  #a = [list(map(float, suba)) for suba in a]
  #final_result = [( ) for places in final_result]
  final_result2 = []
  
  for i in final_result:
    tmp_list = []
    print("i : ",i)
    for j in i:
      print("j : ",j)
      tmp_list.append(place_info[j][0])
    final_result2.append(tmp_list)


  return final_result2



In [22]:
def optimize_course2(place_list,day,db):
  place_info = load_places_location(db, place_list)
  save_id = []  
  result = [[] for _ in range(day)]
  
  AIRPORT = (33.5059364,126.4959513)
  length = len(place_list)
  total_dist = [0]*length
  visit_result = [[] for _ in range(length)]

  dist_matrix = make_dist_matrix_haversine(place_info)

  for i in range(length):
    start = i
    end = 999
    visit = [False] * length
    visit[i] = True    
    now = i
    min_dist = 100
    min_idx = 999
    visit_result[i].append(now)
    print("start : ",start)
    while False in visit: #모든 클러스터를 방문할때까지
      for idx, dist in enumerate(dist_matrix[now]):
        if visit[idx] == True:
          continue
        else:
          if dist < min_dist:
            min_idx = idx
            min_dist = dist
      print("dist from %d to %d : %.2f  "%(now,min_idx,min_dist))
      
      total_dist[start] += min_dist
      now = min_idx
      visit[now] = True
      visit_result[start].append(now)
      min_dist=999
      end = min_idx

    airport_to_start = haversine(AIRPORT,place_info[start][1:]) # 공항에서 시작지점 까지 거리
    end_to_airport = haversine(AIRPORT,place_info[end][1:]) #마지막지점에서 공항 가는 거리
    total_dist[i] += airport_to_start 
    total_dist[i] += end_to_airport  

    print("airport - 시작지점(%s) 까지 거리%s 합산"%(start,airport_to_start))
    print("끝지점(%s) - 공항  까지 거리%s 합산"%(end,end_to_airport))
    print("total_dist : ",total_dist[i])

  shortest_cluster = np.array(total_dist).argsort()[0]
  result_index = visit_result[shortest_cluster]  #최단거리를 보장하는 index 방문 순서 마지막에 이 인덱스랑 Id랑 매핑해줘야함.
 

  #day dividing ...
  day_dividing = []
  print("result_index : ",result_index)

  for comb in combinations(result_index,day-1):
    #comb : (5,3,7)
    #print("comb : ",comb)
    if comb[-1] == result_index[-1]: 
      continue
    dist = 0
    for now in comb:
      next = result_index[result_index.index(now) + 1]
      now_loc = place_info[now][1:]
      next_loc = place_info[next][1:]
      
      dist += haversine(now_loc,next_loc)
    day_dividing.append((dist,comb))
    
  day_dividing.sort(reverse = True) #내림차순 정렬
  
  
  for i in range(len(day_dividing)):
    divider = day_dividing[i][1]
    pass_token = 1
    print("divider : ",divider)
    
    final_result = []
    start = 0
    for i in divider:
      cut = result_index.index(i)  
      final_result.append(result_index[start:cut+1])
      if cut-start > 5: # 하루에 6곳 이상을 방문하는 경우
        pass_token = 0
        break
      start = cut+1

    if pass_token == 0:
      continue
    
    final_result.append(result_index[start:])
  

  #index -> id 전환
    final_result2 = []
    for i in final_result:
      tmp_list = []
      print("i : ",i)
      for j in i:
        print("j : ",j)
        tmp_list.append(place_info[j][0])
      final_result2.append(tmp_list)


    return final_result2



In [12]:
db = load_db()
place_list = [1,3,5,6,7,9,12,18,31]
optimize_course1(place_list,4,db)

making matrix!!!
start :  0
dist from 0 to 7 : 5.83  
dist from 7 to 6 : 9.83  
dist from 6 to 3 : 9.69  
dist from 3 to 1 : 16.95  
dist from 1 to 5 : 3.63  
dist from 5 to 2 : 39.97  
dist from 2 to 4 : 23.24  
dist from 4 to 8 : 10.37  
airport - 시작지점(0) 까지 거리17.43496801230598 합산
끝지점(8) - 공항  까지 거리18.786423002566767 합산
total_dist :  155.73167245618464
start :  1
dist from 1 to 5 : 3.63  
dist from 5 to 3 : 18.16  
dist from 3 to 6 : 9.69  
dist from 6 to 7 : 9.83  
dist from 7 to 0 : 5.83  
dist from 0 to 2 : 19.70  
dist from 2 to 4 : 23.24  
dist from 4 to 8 : 10.37  
airport - 시작지점(1) 까지 거리41.12900821221665 합산
끝지점(8) - 공항  까지 거리18.786423002566767 합산
total_dist :  160.35749408557786
start :  2
dist from 2 to 0 : 19.70  
dist from 0 to 7 : 5.83  
dist from 7 to 6 : 9.83  
dist from 6 to 3 : 9.69  
dist from 3 to 1 : 16.95  
dist from 1 to 5 : 3.63  
dist from 5 to 4 : 53.47  
dist from 4 to 8 : 10.37  
airport - 시작지점(2) 까지 거리29.64061814139395 합산
끝지점(8) - 공항  까지 거리18.786423002566767

[[1, 18, 12, 6], [3, 9], [5], [7, 31]]

##변경 전

In [23]:
place_list = [200,400,94,98,248,174,10,75,112,20,35,38,24, 105,167, 70,92,189, 84, 46,37]
optimize_course1(place_list,6,db)

making matrix!!!
start :  0
dist from 0 to 8 : 2.46  
dist from 8 to 14 : 6.33  
dist from 14 to 1 : 0.39  
dist from 1 to 3 : 9.78  
dist from 3 to 5 : 1.59  
dist from 5 to 2 : 4.63  
dist from 2 to 19 : 13.59  
dist from 19 to 11 : 7.13  
dist from 11 to 12 : 3.20  
dist from 12 to 20 : 6.32  
dist from 20 to 18 : 4.29  
dist from 18 to 16 : 9.38  
dist from 16 to 13 : 13.11  
dist from 13 to 15 : 15.04  
dist from 15 to 7 : 11.69  
dist from 7 to 10 : 0.09  
dist from 10 to 17 : 6.39  
dist from 17 to 9 : 13.08  
dist from 9 to 6 : 23.95  
dist from 6 to 4 : 11.55  
airport - 시작지점(0) 까지 거리26.759442363075724 합산
끝지점(4) - 공항  까지 거리1.7444534347592144 합산
total_dist :  192.4741459150523
start :  1
dist from 1 to 14 : 0.39  
dist from 14 to 0 : 5.96  
dist from 0 to 8 : 2.46  
dist from 8 to 16 : 7.62  
dist from 16 to 19 : 7.58  
dist from 19 to 11 : 7.13  
dist from 11 to 12 : 3.20  
dist from 12 to 20 : 6.32  
dist from 20 to 18 : 4.29  
dist from 18 to 13 : 16.47  
dist from 13 to 15 

[[200, 400, 94, 98, 248, 174, 10, 75, 112, 20, 35, 38, 24],
 [105],
 [167],
 [70, 92, 189],
 [84],
 [46, 37]]

###변경 후

In [24]:
place_list = [200,400,94,98,248,174,10,75,112,20,35,38,24, 105,167, 70,92,189, 84, 46,37]
optimize_course2(place_list,6,db)

making matrix!!!
start :  0
dist from 0 to 8 : 2.46  
dist from 8 to 14 : 6.33  
dist from 14 to 1 : 0.39  
dist from 1 to 3 : 9.78  
dist from 3 to 5 : 1.59  
dist from 5 to 2 : 4.63  
dist from 2 to 19 : 13.59  
dist from 19 to 11 : 7.13  
dist from 11 to 12 : 3.20  
dist from 12 to 20 : 6.32  
dist from 20 to 18 : 4.29  
dist from 18 to 16 : 9.38  
dist from 16 to 13 : 13.11  
dist from 13 to 15 : 15.04  
dist from 15 to 7 : 11.69  
dist from 7 to 10 : 0.09  
dist from 10 to 17 : 6.39  
dist from 17 to 9 : 13.08  
dist from 9 to 6 : 23.95  
dist from 6 to 4 : 11.55  
airport - 시작지점(0) 까지 거리26.759442363075724 합산
끝지점(4) - 공항  까지 거리1.7444534347592144 합산
total_dist :  192.4741459150523
start :  1
dist from 1 to 14 : 0.39  
dist from 14 to 0 : 5.96  
dist from 0 to 8 : 2.46  
dist from 8 to 16 : 7.62  
dist from 16 to 19 : 7.58  
dist from 19 to 11 : 7.13  
dist from 11 to 12 : 3.20  
dist from 12 to 20 : 6.32  
dist from 20 to 18 : 4.29  
dist from 18 to 13 : 16.47  
dist from 13 to 15 

[[200, 400, 94, 98, 248],
 [174, 10, 75, 112, 20],
 [35, 38, 24],
 [105],
 [167, 70, 92, 189, 84],
 [46, 37]]